In [1]:

# libraries
import random
import pandas as pd
import snoop
import feather
import time
import gc
import sys
import feather
import os
import joblib

from varname import nameof
from icecream import ic

from sklearn.datasets import load_digits
from sklearn.feature_selection import SelectKBest, chi2, f_classif, mutual_info_classif
from sklearn.model_selection import cross_val_score, KFold, StratifiedKFold

from sklearn.neural_network import MLPClassifier
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

In [2]:
# Load the model from the file
clf_from_joblib = joblib.load('model_mlpclassifier.pkl')
clf_from_joblib

MLPClassifier(max_iter=1000, random_state=19)

In [3]:
# generate the fake dataset.
nrows = 2000

def make_database(index):
    passengerid = index
    pclass    = int(random.randint(1, 3))
    survived  = int(random.randint(0, 1))
    name      = int(random.randint(0, 1306))
    sex       = int(random.randint(0, 1))
    age       = int(random.randint(2, 80))
    sibsp     = int(random.randint(0, 8))
    parch     = int(random.randint(0, 9))
    ticket    = int(random.randint(0, 928))
    fare      = int(round(random.uniform(0, 513), 4))
    cabin     = int(random.randint(0, 185))
    embarked  = int(random.randint(0, 2))
    boat      = int(random.randint(0, 26))
    body      = int(round(random.uniform(0, 328), 2))
    home_dest = int(random.randint(0,368))
    
    # 'nameof' get name of variables and are appended in one list with column names.
    name_cols = [] # list
    name_cols.append(nameof(passengerid))
    name_cols.append(nameof(pclass))
    name_cols.append(nameof(survived))
    name_cols.append(nameof(name))
    name_cols.append(nameof(sex))
    name_cols.append(nameof(age))
    name_cols.append(nameof(sibsp))
    name_cols.append(nameof(parch))
    name_cols.append(nameof(ticket))
    name_cols.append(nameof(fare))
    name_cols.append(nameof(cabin))
    name_cols.append(nameof(embarked))
    name_cols.append(nameof(boat))
    name_cols.append(nameof(body))
    name_cols.append(nameof(home_dest))
    
    # fake database
    database = {f'passengerid': passengerid,
                f'pclass': pclass,
                f'survived': survived,
                f'name': name,
                f'sex': sex,
                f'age': age,
                f'sibsp': sibsp,
                f'parch': parch,
                f'ticket': ticket,
                f'fare': fare,
                f'cabin': cabin,
                f'embarked': embarked,
                f'boat': boat,
                f'body': body,
                f'home_dest': home_dest
               }
    
    return name_cols, database

def generate_fakedata(nrows):    
    
    irow = 1
    index = 1310
    name_cols, database = make_database(index)
    df = pd.DataFrame(columns=name_cols)
        
    while(irow <= nrows):
        name_cols, database = make_database(index)
        df_database = pd.DataFrame([database])
        df = pd.concat([df, df_database], ignore_index=True)
        irow += 1
        index += 1

    return df

In [4]:
df = generate_fakedata(nrows)

In [5]:
df

,passengerid,pclass,survived,name,sex,age,sibsp,parch,ticket,fare,cabin,embarked,boat,body,home_dest
0,1310,1,0,322,1,63,1,8,146,433,148,2,2,32,209
1,1311,1,1,116,1,6,2,2,877,322,140,2,14,100,32
2,1312,2,1,1047,1,26,0,5,106,163,77,2,8,197,116
3,1313,1,1,412,1,29,1,2,664,172,172,2,12,61,92
4,1314,2,1,543,0,39,0,6,815,79,184,1,17,145,245
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1995,3305,2,0,402,1,5,8,8,345,137,20,2,9,56,305
1996,3306,2,1,910,1,60,1,3,390,490,180,2,2,92,318
1997,3307,1,0,594,1,38,5,0,313,299,5,2,9,274,142
1998,3308,3,0,1078,1,4,7,9,569,92,106,1,0,195,349


In [6]:
# convert object type to category type
df[df.select_dtypes(['object']).columns] = df.select_dtypes(['object']).apply(lambda x: x.astype('int64'))
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2000 entries, 0 to 1999
Data columns (total 15 columns):
 #   Column       Non-Null Count  Dtype
---  ------       --------------  -----
 0   passengerid  2000 non-null   int64
 1   pclass       2000 non-null   int64
 2   survived     2000 non-null   int64
 3   name         2000 non-null   int64
 4   sex          2000 non-null   int64
 5   age          2000 non-null   int64
 6   sibsp        2000 non-null   int64
 7   parch        2000 non-null   int64
 8   ticket       2000 non-null   int64
 9   fare         2000 non-null   int64
 10  cabin        2000 non-null   int64
 11  embarked     2000 non-null   int64
 12  boat         2000 non-null   int64
 13  body         2000 non-null   int64
 14  home_dest    2000 non-null   int64
dtypes: int64(15)
memory usage: 234.5 KB


In [7]:
# Alternatively X and y can be obtained directly from the dataframe attribute:
X = df.drop('survived', axis=1) # drop the column target
y = df['survived'] # target

In [8]:
kf=KFold(n_splits=3)
score=cross_val_score(clf_from_joblib,X,y,cv=kf)

In [9]:
print("Cross Validation Scores are {}".format(score))
print("Average Cross Validation score :{}".format(score.mean()))

Cross Validation Scores are [0.49775112 0.51424288 0.48498498]
Average Cross Validation score :0.4989929959944952
